In [69]:
import pyspark
import os
import pickle

sc = pyspark.SparkContext(appName="BigDataProject")

In [70]:
my_dir = "./BooksDataset"
readFromSavedFile = True

In [71]:
sc

<SparkContext master=local[*] appName=BigDataProject>

In [72]:
#Read Authors

if not(readFromSavedFile):
    authors = []

    for file in os.listdir(my_dir):
        d = os.path.join(my_dir, file)
        if os.path.isdir(d):
            res = file
            authors.append(res)
    print(authors)

    sc.parallelize(authors).saveAsTextFile("./RDD/AuthorsName")


In [73]:
#Read Books per Authors
if not(readFromSavedFile):
    books = [[] for _ in range(len(authors))]


    for i in range(len(authors)):
        mydiraut = my_dir+"/"+authors[i]
        for file in os.listdir(mydiraut):
            d = os.path.join(mydiraut, file)
            books[i].append(file)

    print(books)

    sc.parallelize(books).saveAsTextFile("./RDD/BooksName")

In [74]:
def wordCount(file_path):
    words = sc.textFile(file_path).filter(bool) \
                .flatMap(lambda line: line.split(" ")) \
                .map(lambda w: w.lower()) \
                .map(lambda w: (w, 1)) \
                .reduceByKey(lambda v1, v2: v1 + v2) \
                .map(lambda x: (x[1], x[0])) \
                .sortByKey(False)
    result = words.collect()

    return result


if not(readFromSavedFile):
    allWordsForAnAuthors = []

    for i in range(2,len(authors)):
        for j in range(len(books[i])):
            input_filename = os.path.join(my_dir, authors[i])
            input_filename = os.path.join(input_filename, books[i][j])
            print(input_filename)
            wordsOfABook= wordCount(input_filename)
            
            allWordsForAnAuthors.append(wordsOfABook)


        allWordsForAnAuthors = sc.parallelize(allWordsForAnAuthors)
        allWordsForAnAuthors.flatMap(lambda x : (x[0], x[1])).reduceByKey(lambda v1, v2: v1+v2).sortByKey(False)
        allWordsForAnAuthors.saveAsTextFile('./RDD/'+authors[i])
        
        

        print(f"{authors[i]} {allWordsForAnAuthors.take(100)}")
        allWordsForAnAuthors = []



In [157]:
#Leggo gli rdd
listOfAuthors = sc.textFile("./RDD/AuthorsName/*")
listOfBooks = sc.textFile("./RDD/BooksName/*")

authors = listOfAuthors.collect()
n_authors = len(authors)
#wordsCountAuthors = [[] for _ in range(n_authors)]
wordsCountAuthors = []

for i in range(n_authors):
    print(authors[i])
    tmp = f"./RDD/Books/{authors[i]}/*"
    wordsCountAuthors.append(sc.textFile(tmp))

#wordsCountAuthors = sc.textFile("./RDD/Books/*/*")

Arthur Conan Doyle
Charles Dickens
Daniel Defoe
Edith Wharton
Jane Austen
Joseph Conrad
Lewis Carroll
Louisa May Alcott
Voltaire
William Shakespeare


In [158]:
print(len(wordsCountAuthors))

10


In [160]:
#prova = sc.parallelize(wordsCountAuthors[0])

#print(type(prova))

print(wordsCountAuthors[0].take(10))

TypeError: 'RDD' object is not subscriptable

In [ ]:
#sc.stop()

In [156]:
#Possibile soluzione ?
prova = [[] for i in range(11)]

lista = list(range(10, -1,-1))

rddlist = sc.parallelize(lista)

for i in range(len(lista)):

    prova[i].append(rddlist.collect())


prova[0]
sc.parallelize(prova)

prova[0][0][0]

#print(type(rddlist.collect()))

10